# CS529: Homework #1, Problem 3

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max.columns', 100)
# to draw pictures in jupyter notebook
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns
# we don't like warnings
# you can comment the following 2 lines if you'd like to
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('data.txt')
print("\n".join(data.columns))

 -6.0000000e+000 -1.6416059e+002
